In [6]:
!pip install openpyxl

  Using cached openpyxl-3.0.7-py2.py3-none-any.whl (243 kB)


In [5]:
!pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12075 sha256=cf6514b3af9d8bf260d8657a602c0a319ec50026e472ff914cd1784f92e60a19
  Stored in directory: c:\users\j\appdata\local\pip\cache\wheels\f0\fd\1f\3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar


In [1]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from urllib.request import urlopen
from bs4 import BeautifulSoup

import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [2]:
req = requests.get('https://news.mt.co.kr/mtview.php?no=2021090110084869676')
soup = BeautifulSoup(req.text, 'html.parser')
text2 = soup.find('li', {'class' : 'date'}).text

re.sub(r'[^0-9]', '', text2)[:10]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
res = [0 for i in range(2)]
res[0] = re.sub(r'[^0-9]', '', text2)[:10]

In [ ]:
res[0]

In [3]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [4]:
html = urlopen('http://news.mt.co.kr/mtview.php?no=2021083114085488949')
bsob = BeautifulSoup(html, 'html.parser')
cleanhtml(str(bsob)).replace('\n','')

'샤오미 최고라던 中 소비자들, 삼성 플립3 보더니 \'반전\' - 머니투데이        {            "@context": "http://schema.org",            "@type": "NewsArticle",            "@id": "https://news.mt.co.kr/mtview.php?no=2021083114085488949",            "mainEntityOfPage":{                "@type":"WebPage",                "@id":"https://news.mt.co.kr/mtview.php?no=2021083114085488949"            },            "headline": "샤오미 최고라던 中 소비자들, 삼성 플립3 보더니 반전 - 머니투데이",            "image": {                "@type": "ImageObject",                "@id": "https://menu.mt.co.kr/new_navi/mtlogo223x223.gif",                "url": "https://menu.mt.co.kr/new_navi/mtlogo223x223.gif",                "height": 223,                "width": 223            },            "datePublished": "2021-09-01T05:30:00+09:00",            "dateModified": "2021-09-01T05:30:00+09:00",            "author": {                "@type": "Person",                "name": "이진욱"        },        "publisher": {            "@type": "Organization",            "@id": "

In [5]:
req = requests.get('http://news.mt.co.kr/mtview.php?no=2021083108500081348')
req.encoding = None
soup = BeautifulSoup(req.text, 'html.parser')

In [6]:
####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)


def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    dates = ''
    res = [0 for i in range(2)]
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = cleanhtml(str(soup))
        text2 = soup.find('li', {'class' : 'lasttime'}).text
        dates = re.sub(r'[^0-9]', '', text2)
#         try:
#             text = soup.find('div', {'class' : 'article_view'}).text
#             text2 = soup.find('li', {'class' : 'lasttime'}).text
#             dates = re.sub(r'[^0-9]', '', text2)
#         except:
#             html = urlopen(url)
#             bsob = BeautifulSoup(html, 'html.parser')
#             text = cleanhtml(str(bsob)).replace('\n','')
#             text2 = soup.find('li', {'class' : 'lasttime'}).text
#             dates = re.sub(r'[^0-9]', '', text2)
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 머니투데이
    elif 'news.mt' in url:
        text = soup.find('div', {'class' : 'view_text'}).text
        
    # 그 외
    else:
        text == None
        
    res[0] = text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    res[1] = date[:10]
    
    return res


press_nm = '머니투데이'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = './webdriver/chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)



######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break
          
################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)[0],
                          'date' : crawling_main_text(n_url)[1]}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break
        
#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : 머니투데이


검색할 키워드  :  삼성
수집 뉴스의 수(숫자만 입력) :  10




브라우저를 실행시킵니다(자동 제어)

설정한 언론사를 선택합니다.

<경제/IT> 카테고리에서 <머니투데이>를 찾았으므로 탐색을 종료합니다


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


크롤링을 시작합니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:44<00:00, 10.47s/it]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\j\stock_predict\크롤링\네이버뉴스_본문_10개_삼성_2021-09-02_08시47분.xlsx

결과물의 일부


,title,url,text,date
1,"샤오미 최고라던 中 소비자들, 삼성 플립3 보더니 '반전'",http://news.mt.co.kr/mtview.php?no=20210831140...,12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤럭시Z폴...,2021-09-02
2,8분기 연속 매출 기록 세우는 파운드리 시장…'공격적 투자' 삼성은?,http://news.mt.co.kr/mtview.php?no=20210901161...,삼성전자 화성 반도체 파운드리사업장 EUV(극자외선) V1 생산라인./사진제공=삼성...,2021-09-02
3,'없어서 못 사' 삼성 폴더블폰 돌풍…부품株도 '훨훨',http://news.mt.co.kr/mtview.php?no=20210831161...,"삼성전자 (76,800원 100 0.1%)의 폴더블폰 '갤럭시 Z폴드3'와 '갤럭시...",2021-09-02
4,"14개월만에 노사 합의안 나온 르노삼성, 8월 판매 전년比 16.8%↑",http://news.mt.co.kr/mtview.php?no=20210901155...,르노삼성자동차 XM3르노삼성자동차가 지난 8월 전년 동월 대비 16.8% 증가한 8...,2021-09-02
5,"상속 받은 이재용 부회장, 삼성그룹 지분 70% 늘었다",http://news.mt.co.kr/mtview.php?no=20210901103...,[서울=뉴시스] 고범준 기자 = 이재용 삼성전자 부회장. 2021.08.26. b...,2021-09-02
6,"'갤Z플립' 주문 폭주에…삼성, 부품 공급선 확대 나섰다",http://news.mt.co.kr/mtview.php?no=20210901092...,지난달 12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤...,2021-09-02
7,"삼성전자, 고려대와 친환경 에너지 연구센터 설립 업무협약",http://news.mt.co.kr/mtview.php?no=20210901101...,이기수 삼성전자 생활가전사업부 부사장(왼쪽)과 김용찬 고려대 공과대학장이 지난 8월...,2021-09-02
8,"KH바텍, 삼성전자 폴더블폰 돌풍 수혜…목표가 ↑-한국투자증권",http://news.mt.co.kr/mtview.php?no=20210831085...,12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤럭시Z폴...,2021-09-02
9,"삼성전기, 장기 성장 궤도 진입…목표가 상향-IBK투자증권",http://news.mt.co.kr/mtview.php?no=20210902084...,"MTIR sponsorIBK투자증권은 2일 삼성전기 (185,000원 500 0....",2021-09-02
10,"삼성SDI, LG화학 제치고 배터리 대장주 '왕관'…비결은?",http://news.mt.co.kr/mtview.php?no=20210831160...,"2020년 인터배터리에 참가한 삼성SDI삼성SDI (788,000원 5000 -0....",2021-09-02


In [57]:
 for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)

In [58]:
news_df['url'][1]

'http://news.mt.co.kr/mtview.php?no=2021083114085488949'

In [66]:
crawling_main_text('http://news.mt.co.kr/mtview.php?no=2021083108500081348')[0]

'12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤럭시Z폴드3와 갤럭시Z플립3를 살펴보고 있다. 삼성전자의 갤럭시Z 폴드3와 갤럭시Z플립3는 오는 17일부터 23일까지 국내 사전 판매를 진행하며, 27일 한국, 미국, 유럽을 시작으로 전세계에 순차적으로 출시 예정이다. /사진=이기범 기자 leekb@한국투자증권은 31일 삼성전자의 폴더블폰 신제품이 인기몰이를 하면서 이에 힌지를 공급하는 KH바텍 (30,900원 2850 10.2%)의 수혜가 예상된다며 KH바텍의 목표주가를 기존 3만원에서 3만6000원으로 올렸다. 투자의견은 \'매수\'를 유지했다. KH바텍의 주가는 전날 종가 기준 2만7900원이다. 조철희 연구원은 "삼성전자는 지난 11일 Z폴드3와 Z플립3를 공개했고, 이에 KH바텍의 주가도 이달 들어 26.2% 상승했다"고 말했다.조 연구원은 "삼성전자는 Z폴드3와 Z플립3의 예약판매 수량이 92만대를 기록했다고 밝혔는데 이는 갤럭시 노트 20의 1.3배, S21의 1.8배 수준"이라고 말했다.Z폴드3와 Z플립3이 국내는 물론 중국, 인도, 미국 등에서도 좋은 시장 반응을 얻고 있는 만큼 한국투자증권은 올해 글로벌 폴더블폰 추정 출하량을 기존 890만대에서 990만대로 상향 조정했다. 삼성 신모델 기준으로는 850만대다. 내년 목표 출하량은 기존 1550만대에서 1750만대로 높였다. 조 연구원은 "2023년에는 애플이 폴더블폰 시장에 진입해 시장 규모가 3000만대를 가뿐히 넘길 것"이라며 "삼성전자는 애플의 진출에 앞서 내년 폴더블폰 출하량을 크게 늘려 시장 선점에 나설 것"이라고 예상했다. 이어 "2023년부터 삼성전자와 애플의 양강 체제가 예상된다"며 "애플의 2023년 폴더블폰 첫해 출하량은 1500만대 이상이 될 것"이라고 했다. 조 연구원은 "KH바텍의 올해와 내년 EPS(주당순이익)를 각각 7.5%와 10.5% 올렸다"며 "올해와 내년 영업이익이 각각 전년 대비 591.1%와 103.4% 증가할 것"이라고 말했다